# Prerequisites



In [0]:
storage_account = "azurestgmaveric"
container_name = "users"
storage_account_conf_key = (
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net"
)
storage_account_conf_key_value = "vR3zRwrjFujNOj8lvCqL5E5VV8ifNxpAR8BMvCGuvLjawr10b6zzi3uBq7wNiPvizR1KjlhmaZqW+ASt84bxTA=="
spark.conf.set(storage_account_conf_key, storage_account_conf_key_value)

## Load data into DF

In [0]:
print(users_dataset_path)

abfss://users@azurestgmaveric.dfs.core.windows.net//raw/{users_001.csv}


In [0]:
file_names = "/raw/{users_001.csv}"
users_dataset_path = (
    f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{file_names}"
)
user_df = spark.read.csv(path=users_dataset_path, header=True, inferSchema=True)
print(f"Total no of records:{user_df.count()}")

Total no of records:500


## Create Catalog  and Schema

In [0]:
%sql

-- DROP CATALOG IF EXISTS quickstart_catalog CASCADE;
-- CREATE CATALOG IF NOT EXISTS quickstart_catalog;
DROP SCHEMA IF EXISTS quickstart_catalog.quickstart_schema CASCADE;
CREATE SCHEMA IF NOT EXISTS quickstart_catalog.quickstart_schema;
USE quickstart_catalog.quickstart_schema;

In [0]:
user_df.write.format("delta").mode("append").saveAsTable("user_info")
spark.sql("DESCRIBE FORMATTED user_info")

DataFrame[col_name: string, data_type: string, comment: string]

col_name data_type comment id int null name string null dob date null email string null gender string null country string null region string null city string null asset int null marital_status string null # Delta Statistics Columns Column Names city, name, marital_status, email, dob, country, id, region, asset, gender Column Selection Method first-32 # Detailed Table Information Catalog quickstart_catalog Database quickstart_schema Table user_info Created Time Fri Jul 05 09:49:58 UTC 2024 Last Access UNKNOWN Created By Spark Type MANAGED Location abfss://unity-catalog-storage@dbstoragebaysmlm4fi54k.dfs.core.windows.net/3381240464789137/__unitystorage/catalogs/75fdf4fc-0ac1-4ae7-bda5-690e9166e26c/tables/e6baae51-5325-43d0-8a58-2d7c50d746f1 Provider delta Owner naveen_1719215901936@npmavericsystems.onmicrosoft.com Is_managed_location true Table Properties [delta.enableDeletionVectors=true,delta.feature.deletionVectors=supported,delta.minReaderVersion=3,delta.minWriterVersion=7]

## Create Delta Table and Load

In [0]:
user_df.write.format("delta").mode("append").saveAsTable("user_info")
spark.sql("DESCRIBE FORMATTED user_info").display()

## Read Delta Table

In [0]:
spark.read.table("user_info").limit(3).display()

id name dob email gender country region city asset marital_status 1 Heather Gibbs 2024-10-31 heathergibbs6243@gmail.com Female United States Virginia Virginia Beach 734388 Married 2 Herrod Petersen 2024-02-19 herrodpetersen@yahoomail.com Male United States Arizona Phoenix 113506 Single 3 Ocean Workman 2024-10-10 oceanworkman2328@ymail.com Male United States Tennessee Clarksville 139985 Married

In [0]:
%sql

SELECT count(*) FROM user_info;

count(1) 500

# Source with More Columns

## Read Data with More COlumns

In [0]:
file_names = "/raw/{users_006_new_column_education.csv}"
users_dataset_path = (
    f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{file_names}"
)
df = spark.read.csv(path=users_dataset_path, header=True, inferSchema=True)
print(f"Total no of records:{df.count()}")

Total no of records:500


## Write Data with more column into Delta Table

In [0]:
df.write.format("delta").mode("append").saveAsTable("user_info")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-984221041466077>, line 1
----> 1 df.write.format("delta").mode("append").saveAsTable("user_info")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1856, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1854 if format is not None:
   1855     self.format(format)
-> 1856 self._jwrite.saveAsTable(name)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(self, *arg

## Solution 

In [0]:
df.write.format("delta").mode("append").option("mergeSchema","true").saveAsTable("user_info")

In [0]:
%sql
select count(*) from user_info;

count(1) 1000

In [0]:
spark.sql("select * from user_info where id > 500").limit(10).display()

id name dob email gender country region city asset marital_status education 2501 Macy Holcomb 2024-10-20 macyholcomb@yahoomail.com Transgender India Karnataka Bidar 455582 Common Law Bachelor's Degree 2502 Zeph Shepherd 2024-03-10 zephshepherd5778@ymail.com Transgender India Dadra and Nagar Haveli Silvassa 480168 Common Law Bachelor's Degree 2503 Elton Stark 2025-03-23 eltonstark7778@yahoomail.com Female India Tamil Nadu Ambattur 428049 Single Bachelor's Degree 2504 Alisa Cook 2025-03-19 alisacook4349@gmail.com Male United States Maine Bangor 594097 Common Law Bachelor's Degree 2505 Boris Snyder 2024-08-03 borissnyder2540@yahoomail.com Male United States Louisiana Baton Rouge 977312 Divorced Bachelor's Degree 2506 Whilemina Olsen 2024-09-29 whileminaolsen8928@yahoomail.com Female United States Arkansas Fort Smith 612058 Common Law Postgraduate Degree 2507 Kalia Frye 2024-09-20 kaliafrye@ymail.com Female United States Washington Bellevue 135753 Common Law Bachelor's Degree 2508 Mollie Shelton 2025-01-29 mollieshelton4280@gmail.com Female India Chhattisgarh Raj Nandgaon 826113 Divorced Postgraduate Degree 2509 Edan Chaney 2024-09-30 edanchaney610@ymail.com Transgender India Bihar Patna 845700 Divorced Postgraduate Degree 2510 Rebecca Kane 2024-08-12 rebeccakane4109@ymail.com Transgender United States Delaware Bear 633634 Married Postgraduate Degree

# Source with less columns

In [0]:
file_names = "/raw/{users_012_less_columns.csv}"
users_dataset_path = (
    f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{file_names}"
)
user_df_with_less_columns = spark.read.csv(path=users_dataset_path, header=True, inferSchema=True)
user_df_with_less_columns.write.format("delta").mode("append").saveAsTable("user_info")


Total no of records:500


In [0]:
spark.read.table("user_info").count()

1500

# Source with different Data type